# Pre-Processing
For each data source, missing values are imputed by taking the mean. Additionally, the values for each attribute/column are corrected by removing  unnecessary characters, e.g. extra parentheses. The number of NaN or Null values is checked per attribute/column, at least 30% non-null or NaN rows/values should be available else the column/attribute will be dropped. Also, some attributes are dropped that are considered to be irrelevant/useless for the project. Each data source is transformed to a long format in order to get the table that represents the knowledge graph for this project. Also, the values of attributes or columns of data sources that contain data for multiple years (2018-2021) will be averaged.

In [1]:
# IMPORTS
import pandas as pd
import os
import glob
import csv
import numpy as np

### Lookup table Municipality - Province
A table that shows which municipality belongs to which province. All unique rows/combinations of the years 2018, 2019, 2020, and 2021 are taken into account and thus combined into a single dataframe/table because in this project all the data sources will be related to these years (or partially since not all data sources have data for every year)

Packages that are needed in order for this code to run (command to run in a cell: !pip install package-name):
- XLRD

In [2]:
# classifications of municipalities by province in the Netherlands 
df_2021 = pd.read_excel('data/Source_0/gemeenten-alfabetisch-2021.xlsx')[['Gemeentenaam', 'Provincienaam']]
df_2020 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2020.xlsx')[['Gemeentenaam', 'Provincienaam']]
# for 2018 and 2019 files, you need pip install xlrd
df_2019 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2019.xls')[['Gemeentenaam', 'Provincienaam']]
df_2018 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2018.xls')[['Gemeentenaam', 'Provincienaam']]

# 2020 and 2019 are exactly the same
# combine 2020 and 2018 in order to get all unique rows from both data sources
df_combi1820 = df_2018.merge(df_2019, on=['Gemeentenaam', 'Provincienaam'], how='outer')

# combine 2021 and the newly created 2018-2020 dataframe
df_final = df_combi1820.merge(df_2021, on=['Gemeentenaam', 'Provincienaam'], how='outer')

# Fryslan is kept instead of Friesland because in ALL data only the first one is used. 
df_final = df_final[df_final["Provincienaam"]!="Friesland"]
df_final

,Gemeentenaam,Provincienaam
0,Aa en Hunze,Drenthe
1,Aalburg,Noord-Brabant
2,Aalsmeer,Noord-Holland
3,Aalten,Gelderland
5,Alblasserdam,Zuid-Holland
...,...,...
403,Terschelling,Fryslân
404,Tytsjerksteradiel,Fryslân
405,Vlieland,Fryslân
406,Waadhoeke,Fryslân


### Source 1 preprocessing
Contains data for 2021 only

In [3]:
# Source file cleaning
source_number = 1
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_1 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_1.columns = source_1.columns.str[1:]
source_1.columns = source_1.columns.str[:-1]
source_1["Wijken en buurten"] = source_1["Wijken en buurten"].str[1:]
source_1["Wijken en buurten"] = source_1["Wijken en buurten"].str[:-1]
source_1["Regioaanduiding/Soort regio (omschrijving)"] = source_1["Regioaanduiding/Soort regio (omschrijving)"].str[1:]
source_1["Regioaanduiding/Soort regio (omschrijving)"] = source_1["Regioaanduiding/Soort regio (omschrijving)"].str[:-1]

# filter only municipalities
source_1 = source_1[source_1["Regioaanduiding/Soort regio (omschrijving)"]=="Gemeente  "]
# rename column for merging
source_1.rename(columns={"Wijken en buurten": "Gemeentenaam"}, inplace = True)

# drop column
source_1.drop(columns={"Regioaanduiding/Soort regio (omschrijving)"}, inplace=True)

# to check:
# source_1.columns
# show nr of non-null values in each column
# source_1.count()

# merge
merged_source_1 = pd.merge(df_final, source_1, on = "Gemeentenaam", how='right')

# inspect rows where province is null
merged_source_1[merged_source_1["Provincienaam"].isnull()]

# drop rows where province is nan
merged_source_1 = merged_source_1[merged_source_1["Provincienaam"].notnull()]

# require 30% non-null columns, more than this drops important columns
limitPer = len(merged_source_1) * 0.3
merged_source_1 = merged_source_1.dropna(thresh=limitPer, axis=1)

# KE table - Subject Predicate Object tuples
# Renaming the columns, first col is considered to be of type subject and the others of type predicate
source_1_wide_format = merged_source_1.rename(
    columns={"Gemeentenaam":"Subject", "Provincienaam":"isPartOf", "Bevolking/Aantal inwoners (aantal)":"hasPopulationSize(2021)", \
    "Bevolking/Geslacht/Mannen (aantal)":"hasNumberOfMen(2021)", "Bevolking/Geslacht/Vrouwen (aantal)":"hasNumberOfWomen(2021)", \
    "Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-0-15(2021)", \
    "Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-15-25(2021)", \
    "Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-25-45(2021)", \
    "Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-45-65(2021)", \
    "Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal)":"hasNumberOfPeopleInAgeGroup-65+(2021)", \
    "Bevolking/Burgerlijke staat/Ongehuwd (aantal)":"hasNumberOfUnmarried(2021)", \
    "Bevolking/Burgerlijke staat/Gehuwd (aantal)":"hasNumberOfMarried(2021)", \
    "Bevolking/Burgerlijke staat/Gescheiden (aantal)":"hasNumberOfDivorced(2021)", \
    "Bevolking/Burgerlijke staat/Verweduwd (aantal)":"hasNumberOfWidowed(2021)", \
    "Bevolking/Bevolkingsdichtheid (aantal inwoners per km²)":"hasPopulationDensityPerKm²(2021)"})

# Change wide format to long format in order to get the KE table
long_format_source_1 = source_1_wide_format.melt(id_vars=["Subject"], var_name="Predicate", value_name="Object")
long_format_source_1


,Subject,Predicate,Object
0,Aa en Hunze,isPartOf,Drenthe
1,Aalsmeer,isPartOf,Noord-Holland
2,Aalten,isPartOf,Gelderland
3,Achtkarspelen,isPartOf,Fryslân
4,Alblasserdam,isPartOf,Zuid-Holland
...,...,...,...
4923,Zundert,hasPopulationDensityPerKm²(2021),182.0
4924,Zutphen,hasPopulationDensityPerKm²(2021),1175.0
4925,Zwartewaterland,hasPopulationDensityPerKm²(2021),277.0
4926,Zwijndrecht,hasPopulationDensityPerKm²(2021),2206.0


### Source 2 preprocessing
Contains data for 2019-2021 only

In [4]:
source_number = 2
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_2 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_2.columns = source_2.columns.str[1:]
source_2.columns = source_2.columns.str[:-1]
source_2["Regio's"] = source_2["Regio's"].str[1:]
source_2["Regio's"] = source_2["Regio's"].str[:-1]
source_2["Perioden"] = source_2["Perioden"].str[1:]
source_2["Perioden"] = source_2["Perioden"].str[:-1]
# rename column for merging
source_2.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

# to check:
# show nr of non-null values in each column
# source_2.count()

merged_source_2 = pd.merge(df_final, source_2, on = "Gemeentenaam", how='right')

# inspect rows where province is null
merged_source_2[merged_source_2["Provincienaam"].isnull()]

# drop rows where province is nan
merged_source_2 = merged_source_2[merged_source_2["Provincienaam"].notnull()]

# require 30% non-null columns, more than this drops important columns
limitPer = len(merged_source_2) * 0.3
merged_source_2 = merged_source_2.dropna(thresh=limitPer, axis=1)

# replace , with . in strings that are supposed to be numbers
# then convert those columns to numeric

# get the non-numeric columns, except the first 3
columns_to_convert = merged_source_2.select_dtypes(exclude=[np.number]).iloc[:,3:]
# convert
for column in columns_to_convert:
    merged_source_2[column] = merged_source_2[column].str.replace(',','.')
    merged_source_2[column] = pd.to_numeric(merged_source_2[column])

# IMPUTE MISSING VALUES
# get list of all provinces
provinces = merged_source_2['Provincienaam'].unique().tolist()

# for each province
for province in provinces:
    province_data = merged_source_2[merged_source_2['Provincienaam']==province]
    # get only numeric columns from province_data
    imputed_subset = province_data.iloc[:, 3:]
    # impute
    imputed_subset = imputed_subset.fillna(imputed_subset.mean())
    # replace with imputed data
    province_data.iloc[:, 3:] = imputed_subset
    merged_source_2[merged_source_2['Provincienaam']==province] = province_data

# AVERAGE the years
merged_source_2 = merged_source_2.groupby(['Gemeentenaam', 'Provincienaam']).mean().reset_index()

# KE table - Subject Predicate Object tuples
# Renaming the columns, first col is considered to be of type subject and the others of type predicate
source_2_wide_format = merged_source_2.rename(columns={"Gemeentenaam":"Subject", "Provincienaam":"isPartOf",\
        "Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)":"hasPopulationSize(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal)":"hasNumberOfMen(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal)":"hasNumberOfWomen(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-5-(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-5-10(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal)":"hasNumberOfPeopleInAgeGroup-10-15(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal)": "hasNumberOfPeopleInAgeGroup-15-20(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/20 tot 25 jaar (aantal)": "hasNumberOfPeopleInAgeGroup-20-25(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/25 tot 45 jaar (aantal)": "hasNumberOfPeopleInAgeGroup-25-45(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/45 tot 65 jaar (aantal)": "hasNumberOfPeopleInAgeGroup-45-65(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/65 tot 80 jaar (aantal)": "hasNumberOfPeopleInAgeGroup-65-80(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/80 jaar of ouder (aantal)": "hasNumberOfPeopleInAgeGroup-80+(avg)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/Jonger dan 5 jaar (%)": "hasRelativePeopleInAgeGroup-5-(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/5 tot 10 jaar (%)": "hasRelativePeopleInAgeGroup-5-10(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/10 tot 15 jaar (%)": "hasRelativePeopleInAgeGroup-10-15(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/15 tot 20 jaar (%)": "hasRelativePeopleInAgeGroup-15-20(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/20 tot 25 jaar (%)": "hasRelativePeopleInAgeGroup-20-25(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/25 tot 45 jaar (%)": "hasRelativePeopleInAgeGroup-25-45(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/45 tot 65 jaar (%)": "hasRelativePeopleInAgeGroup-45-65(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/65 tot 80 jaar (%)": "hasRelativePeopleInAgeGroup-65-80(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen, relatief/80 jaar of ouder (%)": "hasRelativePeopleInAgeGroup-80+(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Demografische druk/Totale druk (%)":"hasTotalDemographicPressure(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Demografische druk/Groene druk (%)":"hasGreenDemographicPressure(avg %)", \
        "Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Demografische druk/Grijze druk (%)":"hasGreyDemographicPressure(avg %)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Geboorte (aantal)":"hasNumberOfBirth(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Geboorte, relatief (per 1 000 inwoners)":"hasRelativeNumberOfBirth(avg per 1000 residents)", 
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Sterfte (aantal)":"hasNumberOfDeath(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Sterfte, relatief (per 1 000 inwoners)":"hasRelativeNumberOfDeath(avg per 1000 residents)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Geboorteoverschot (aantal)":"hasBirthSurplusOf(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Geboorteoverschot, relatief (per 1 000 inwoners)":"hasRelativeBirthSurplusOf(avg per 1000 residents)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Doodsoorzaken/Nieuwvormingen (aantal)":"hasNewFormationsOf(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Doodsoorzaken/Ziekten van hart en vaatstelsel (aantal)":"hasNumberOfCardiovascularDisease(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Doodsoorzaken/Ziekten van ademhalingsstelsel (aantal)":"hasNumberOfRespiratorySystemDisease(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Doodsoorzaken/Uitwendige doodsoorzaken (aantal)":"hasExternalCausesOfDeathOf(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Geboorte en sterfte/Doodsoorzaken/Overige doodsoorzaken (aantal)":"hasOtherCausesOfDeathOf(avg)", \
        "Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Bevolkingsgroei, relatief (per 1 000 inwoners)":"hasRelativePopulationGrowthOf(avg per 1 000 residents)", \
        "Onderwijs/Naar woongemeente/Gediplomeerden/Voortgezet onderwijs (aantal)":"hasGraduatesOfSecondaryEducationOf(avg)", \
        "Onderwijs/Naar woongemeente/Gediplomeerden/Middelbaar beroepsonderwijs (aantal)":"hasGraduatesOfPost-SecondaryVocationalEducationOf(avg)", \
        "Onderwijs/Naar woongemeente/Gediplomeerden/Hoger beroepsonderwijs bachelor (aantal)":"hasGraduatesOfHigherVocationalEducationOf(avg)", \
        "Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal)":"hasGraduatesOfDoctorateOf(avg)", \
        "Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)":"hasJobsInA-AgricultureForestryFishingOf(avg %)", \
        "Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)":"hasJobsInIndustryEnergyOf(avg %)", \
        "Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)":"hasJobsInCommercialServiceOf(avg %)", \
        "Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)":"hasJobsInNon-CommercialServeOf(avg %)", \
        "Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km)": "hasDistanceToGPPracticeOf(avg km)", \
        "Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal)":"hasNumberOfGPPracticesWithin3KmOf(avg)", \
        "Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km)":"hasDistanceToGeneralPracticeCenterOf(avg km)", \
        "Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km)":"hasDistanceToHospitalOf(avg km)", \
        "Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)":"hasNumberOfHospitalsWithin20KmOf(avg)"})

# Change wide format to long format in order to get the KE table
long_format_source_2 = source_2_wide_format.melt(id_vars=["Subject"], var_name="Predicate", value_name="Object")
long_format_source_2

C:\Users\t1rk9\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,Subject,Predicate,Object
0,'s-Hertogenbosch,isPartOf,Noord-Brabant
1,Aa en Hunze,isPartOf,Drenthe
2,Aalburg,isPartOf,Noord-Brabant
3,Aalsmeer,isPartOf,Noord-Holland
4,Aalten,isPartOf,Gelderland
...,...,...,...
18895,Zundert,hasNumberOfHospitalsWithin20KmOf(avg),2.474731
18896,Zutphen,hasNumberOfHospitalsWithin20KmOf(avg),1.962092
18897,Zwartewaterland,hasNumberOfHospitalsWithin20KmOf(avg),1.794444
18898,Zwijndrecht,hasNumberOfHospitalsWithin20KmOf(avg),8.306667


### Source 4 preprocessing
Contains data for 2019, 2020 and 2021 only

In [5]:
source_number = 4
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_4 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

# remove "" from column names and string values
source_4.columns = source_4.columns.str[1:]
source_4.columns = source_4.columns.str[:-1]
source_4["Regio's"] = source_4["Regio's"].str[1:]
source_4["Regio's"] = source_4["Regio's"].str[:-1]
source_4["Perioden"] = source_4["Perioden"].str[1:]
source_4["Perioden"] = source_4["Perioden"].str[:-1]
# rename column for merging
source_4.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

# drop columns with only 1 value
for col in source_4.columns:
    if len(source_4[col].unique()) == 1:
        print(col)
        source_4.drop(col,inplace=True,axis=1)

# to check
# show nr of non-null values in each column
# source_4.count()

merged_source_4 = pd.merge(df_final, source_4, on = "Gemeentenaam", how='right')

# inspect rows where province is null
merged_source_4[merged_source_4["Provincienaam"].isnull()]

# drop rows where province is nan
merged_source_4 = merged_source_4[merged_source_4["Provincienaam"].notnull()]

# IMPUTE MISSING VALUES:
# get list of all provinces
provinces = merged_source_4['Provincienaam'].unique().tolist()

# for each province
for province in provinces:
    province_data = merged_source_4[merged_source_4['Provincienaam']==province]
    # get only numeric columns from province_data
    imbuted_subset = province_data.iloc[:, 2:]
    # impute
    imbuted_subset = imbuted_subset.fillna(imbuted_subset.mean())
    # replace with imputed data
    province_data.iloc[:, 2:] = imbuted_subset
    merged_source_4[merged_source_4['Provincienaam']==province] = province_data
merged_source_4

# AVERAGE the years
merged_source_4 = merged_source_4.groupby(['Gemeentenaam', 'Provincienaam']).mean().reset_index()

# KE table - Subject Predicate Object tuples
# Renaming the columns, first col is considered to be of type subject and the others of type predicate
source_4_wide_format = merged_source_4.rename(columns={"Gemeentenaam":"Subject", "Provincienaam":"isPartOf", \
    "Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal)":"hasNumberOfMen(avg)", \
    "Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal)":"hasNumberOfMenInAgeGroup-20-(avg)", \
    "Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal)":"hasNumberOfMenInAgeGroup-20-65(avg)", \
    "Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal)":"hasNumberOfMenInAgeGroup-65-75(avg)", \
    "Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal)":"hasNumberOfMenInAgeGroup-75-85(avg)", \
    "Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal)":"hasNumberOfMenInAgeGroup-85+(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal)":"hasNumberOfWomen(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal)":"hasNumberOfWomenInAgeGroup-20-(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal)":"hasNumberOfWomenInAgeGroup-20-65(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal)":"hasNumberOfWomenInAgeGroup-65-75(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal)":"hasNumberOfWomenInAgeGroup-75-85(avg)", \
    "Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal)":"hasNumberOfWomenInAgeGroup-85+(avg)", \
    "Minderheden (aantal)":"hasNumberOfMinoritiesOf(avg)",\
    "Inwoners in zorginstellingen (aantal)":"hasNumberOfResidentsInCareInstitutions(avg)"})

# Change wide format to long format in order to get the KE table
long_format_source_4 = source_4_wide_format.melt(id_vars=["Subject"], var_name="Predicate", value_name="Object")
long_format_source_4

Status cijfer
Banen van werknemers (x 1 000)
Provinciale wegen (km)


C:\Users\t1rk9\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


,Subject,Predicate,Object
0,'s-Hertogenbosch,isPartOf,Noord-Brabant
1,Aa en Hunze,isPartOf,Drenthe
2,Aalburg,isPartOf,Noord-Brabant
3,Aalsmeer,isPartOf,Noord-Holland
4,Aalten,isPartOf,Gelderland
...,...,...,...
5665,Zundert,hasNumberOfResidentsInCareInstitutions(avg),413.333333
5666,Zutphen,hasNumberOfResidentsInCareInstitutions(avg),853.333333
5667,Zwartewaterland,hasNumberOfResidentsInCareInstitutions(avg),250.0
5668,Zwijndrecht,hasNumberOfResidentsInCareInstitutions(avg),400.0


### Source 5 preprocessing
Contains data for 2019, 2020 and 2021 only

In [6]:
source_number = 5
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_5 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

# remove "" from column names and string values
source_5.columns = source_5.columns.str[1:]
source_5.columns = source_5.columns.str[:-1]

# remove "" and " (PV)" from column values
source_5["Regio's"] = source_5["Regio's"].str[1:]
source_5["Regio's"] = source_5["Regio's"].str[:-6]
source_5["Beroepen en specialismen"] = source_5["Beroepen en specialismen"].str[1:]
source_5["Beroepen en specialismen"] = source_5["Beroepen en specialismen"].str[:-1]
source_5["Perioden"] = source_5["Perioden"].str[1:]
source_5["Perioden"] = source_5["Perioden"].str[:-1]

# remove * char from 2020
source_5["Perioden"] = source_5["Perioden"].str.replace("*","")

# drop age, social columns
source_5.drop("Leeftijd",inplace=True,axis=1)
source_5.drop("Sociaaleconomische categorie",inplace=True,axis=1)

# inspect rows where province is null
source_5[source_5["Regio's"].isnull()]

# drop rows where province is nan
source_5 = source_5[source_5["Regio's"].notnull()]

# AVERAGE the years
source_5 = source_5.groupby(["Beroepen en specialismen", "Regio's"]).mean().reset_index()

# KE table - Subject Predicate Object tuples
# Renaming the columns, first col is considered to be of type subject and the others of type predicate
source_5_wide_format = source_5.rename(columns={"Regio's":"Subject", "Beroepen en specialismen":"hasJobOrSpecialism", \
    "Medisch geschoolden (aantal)":"hasNumberOfMedicalProfessionalsOf(avg)"})

# Change wide format to long format in order to get the KE table
long_format_source_5 = source_5_wide_format.melt(id_vars=["Subject"], var_name="Predicate", value_name="Object")
long_format_source_5

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/398666629.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_5["Perioden"] = source_5["Perioden"].str.replace("*","")
C:\Users\t1rk9\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Subject,Predicate,Object
0,Drenthe,hasJobOrSpecialism,Apotheker
1,Flevoland,hasJobOrSpecialism,Apotheker
2,Fryslân,hasJobOrSpecialism,Apotheker
3,Gelderland,hasJobOrSpecialism,Apotheker
4,Groningen,hasJobOrSpecialism,Apotheker
5,Limburg,hasJobOrSpecialism,Apotheker
6,Noord-Brabant,hasJobOrSpecialism,Apotheker
7,Noord-Holland,hasJobOrSpecialism,Apotheker
8,Overijssel,hasJobOrSpecialism,Apotheker
9,Utrecht,hasJobOrSpecialism,Apotheker


### Source 6 preprocessing
Contains data for 2019, 2020 and 2021 only

In [7]:
source_number = 6
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_6 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_6.columns = source_6.columns.str[1:]
source_6.columns = source_6.columns.str[:-1]
source_6["Regio's"] = source_6["Regio's"].str[1:]
source_6["Regio's"] = source_6["Regio's"].str[:-1]
source_6["Perioden"] = source_6["Perioden"].str[1:]
source_6["Perioden"] = source_6["Perioden"].str[:-1]
source_6['Donorregistratie'] = source_6['Donorregistratie'].str[1:]
source_6['Donorregistratie'] = source_6['Donorregistratie'].str[:-1]

# remove * char from 2021
source_6["Perioden"] = source_6["Perioden"].str.replace("*","")

# rename column for merging
source_6.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

# to check
# source_6.count()

# merge
merged_source_6 = pd.merge(df_final, source_6, on = "Gemeentenaam", how='right')

# inspect rows where province is null
merged_source_6[merged_source_6["Provincienaam"].isnull()]

# drop rows where province is nan
merged_source_6 = merged_source_6[merged_source_6["Provincienaam"].notnull()]

# replace , with . in strings that are supposed to be numbers
# then convert those columns to numeric

# get the non-numeric columns, except the first 4
columns_to_convert = merged_source_6.select_dtypes(exclude=[np.number]).iloc[:,4:]
# convert
for column in columns_to_convert:
    merged_source_6[column] = merged_source_6[column].str.replace(',','.')
    merged_source_6[column] = pd.to_numeric(merged_source_6[column])
    
# find the mean for for all years
condition = merged_source_6.groupby(['Gemeentenaam', 'Provincienaam', 'Donorregistratie'])
merged_source_6 = condition.mean('Donorregister, vastgelegde keuze (x 1 000)').reset_index()

# Translate
merged_source_6.columns = ['Municipality', 'Province', 'Donor registration', 'Donor register, fixed choice (x 1,000)']
merged_source_6["Donor registration"].replace({"Geregistreerd in donorregister": "Registered in donor register", "Niet geregistreerd in donorregister": "Not registered in donor register", "Totaal wel of niet geregistreerd":"Total registered or not registered"}, inplace=True)

# subject-predicate-object format
s6_dict = {'Subject':[] , 'Predicate': [], 'Object': []}

subjects = merged_source_6['Municipality'].unique()
predicates = [list(merged_source_6.columns)[1]]
for item in list(merged_source_6['Donor registration'].unique()):
    predicates.append(item)

for subject in subjects:
    s6_dict['Subject'].append(subject)
    s6_dict['Predicate'].append(predicates[0])
    s6_dict['Object'].append(merged_source_6[merged_source_6['Municipality']==subject]['Province'].unique()[0])
    
    for pedictate in predicates[1:]:
        s6_dict['Subject'].append(subject)
        s6_dict['Predicate'].append(pedictate)
        intermediate = merged_source_6[merged_source_6['Municipality']==subject]
        value = intermediate[intermediate['Donor registration']==pedictate]['Donor register, fixed choice (x 1,000)']
        s6_dict['Object'].append(float(value))
        
spo_6 = pd.DataFrame(data=s6_dict)
donor_ids = np.where(spo_6['Predicate']!='Province')

for donor_id in donor_ids:
    spo_6['Predicate'][donor_id] = spo_6['Predicate'][donor_id] + ' (x 1000)'
    
spo_6

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/919591505.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_6["Perioden"] = source_6["Perioden"].str.replace("*","")


,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
1,'s-Hertogenbosch,Registered in donor register (x 1000),85.3
2,'s-Hertogenbosch,Not registered in donor register (x 1000),50.8
3,'s-Hertogenbosch,Total registered or not registered (x 1000),136.066667
4,Aa en Hunze,Province,Drenthe
...,...,...,...
1367,Zwijndrecht,Total registered or not registered (x 1000),38.966667
1368,Zwolle,Province,Overijssel
1369,Zwolle,Registered in donor register (x 1000),69.1
1370,Zwolle,Not registered in donor register (x 1000),41.733333


### Source 7 preprocessing
Contains data for 2019 and 2020 only

In [8]:
source_number = 7
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_7 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")
source_7.columns = source_7.columns.str[1:]
source_7.columns = source_7.columns.str[:-1]
source_7.rename(columns={'"Geslacht': 'Geslacht'}, inplace=True)

source_7["Geslacht"] = source_7["Geslacht"].str[1:]
source_7["Geslacht"] = source_7["Geslacht"].str[:-1]

source_7["Leeftijd"] = source_7["Leeftijd"].str[1:]
source_7["Leeftijd"] = source_7["Leeftijd"].str[:-1]

source_7["Geneesmiddelengroep (ATC)"] = source_7["Geneesmiddelengroep (ATC)"].str[1:]
source_7["Geneesmiddelengroep (ATC)"] = source_7["Geneesmiddelengroep (ATC)"].str[:-1]

source_7["Perioden"] = source_7["Perioden"].str[1:]
source_7["Perioden"] = source_7["Perioden"].str[:-1]
source_7["Perioden"] = source_7["Perioden"].str.replace("*","")

source_7["Regio's"] = source_7["Regio's"].str[1:]
source_7["Regio's"] = source_7["Regio's"].str[:-1]

source_7["Personen met geneesmiddelen, relatief (%)"] = source_7["Personen met geneesmiddelen, relatief (%)"].str[1:]
source_7["Personen met geneesmiddelen, relatief (%)"] = source_7["Personen met geneesmiddelen, relatief (%)"].str[:-1]

# drop columns with only 1 value
for col in source_7.columns:
    if len(source_7[col].unique()) == 1:
        source_7.drop(col,inplace=True,axis=1)

source_7.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)
merged_source_7 = pd.merge(df_final, source_7, on = "Gemeentenaam", how='right')

# some municipalities dont map to provinces, drop those
merged_source_7.drop(merged_source_7[merged_source_7['Provincienaam'].isnull()].index, inplace=True)

merged_source_7['Personen met geneesmiddelen, relatief (%)'] = merged_source_7['Personen met geneesmiddelen, relatief (%)'].str.replace(',','.')
merged_source_7['Personen met geneesmiddelen, relatief (%)'] = pd.to_numeric(merged_source_7['Personen met geneesmiddelen, relatief (%)'])

#impute
means = {}
for provincienaam in merged_source_7['Provincienaam'].unique():
    means[provincienaam] = merged_source_7[merged_source_7['Provincienaam'] == provincienaam]['Personen met geneesmiddelen, relatief (%)'].mean()

for i, row in merged_source_7[merged_source_7['Personen met geneesmiddelen, relatief (%)'].isnull()].iterrows():
    merged_source_7.at[i, 'Personen met geneesmiddelen, relatief (%)'] = means[row['Provincienaam']]

# to check:
# source_7.count()
# merged_source_7.count()

#compute mean for all years
condition = merged_source_7.groupby(['Gemeentenaam', 'Provincienaam', 'Geslacht', 'Leeftijd'])
merged_source_7 = condition.mean('Personen met geneesmiddelen, relatief (%)').reset_index()

# Translate
merged_source_7.columns = ['Municipality', 'Province', 'Gender', 'Age group', 'Persons with medicines - relative (%)']
merged_source_7["Gender"].replace({"Mannen": "Men", "Vrouwen": "Women"}, inplace=True)
merged_source_7["Age group"].replace({"0 tot 15 jaar": "0 to 15 years", "15 tot 25 jaar": "15 to 25 years", "25 tot 35 jaar":"25 to 35 years", "35 tot 45 jaar":"35 to 45 years", "45 tot 55 jaar":"45 to 55 years", "55 tot 65 jaar":"55 to 65 years", "65 tot 75 jaar":"65 to 75 years", "75 jaar of ouder":"75 years or older"}, inplace=True)

# subject-predicate-object format

s7_dict = {'Subject':[] , 'Predicate': [], 'Object': []}

subjects = merged_source_7['Municipality'].unique()

predicates = [list(merged_source_7.columns)[1]]
genders = list(merged_source_7['Gender'].unique())
ages = list(merged_source_7['Age group'].unique())

for gender in genders:
    for age in ages:
        predicates.append('Persons with medicines - relative (%)'+', '+gender+', '+age)


for subject in subjects:
    s7_dict['Subject'].append(subject)
    s7_dict['Predicate'].append(predicates[0])
    s7_dict['Object'].append(merged_source_7[merged_source_7['Municipality']==subject]['Province'].unique()[0])
    
    for pedictate in predicates[1:]:
        s7_dict['Subject'].append(subject)
        s7_dict['Predicate'].append(pedictate)
        intermediate = merged_source_7[merged_source_7['Municipality']==subject]
        intermediate = intermediate[intermediate['Gender']==pedictate.split(', ')[1]]
        value = intermediate[intermediate['Age group']==pedictate.split(', ')[2]]['Persons with medicines - relative (%)']
        s7_dict['Object'].append(float(value))
        
spo_7 = pd.DataFrame(data=s7_dict)
spo_7

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/2198491845.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_7["Perioden"] = source_7["Perioden"].str.replace("*","")


,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
1,'s-Hertogenbosch,"Persons with medicines - relative (%), Men, 0 ...",3.458367
2,'s-Hertogenbosch,"Persons with medicines - relative (%), Men, 15...",4.7
3,'s-Hertogenbosch,"Persons with medicines - relative (%), Men, 25...",5.35
4,'s-Hertogenbosch,"Persons with medicines - relative (%), Men, 35...",1.95
...,...,...,...
6404,Zwolle,"Persons with medicines - relative (%), Women, ...",4.0
6405,Zwolle,"Persons with medicines - relative (%), Women, ...",8.45
6406,Zwolle,"Persons with medicines - relative (%), Women, ...",6.6
6407,Zwolle,"Persons with medicines - relative (%), Women, ...",4.45


### Source 9 preprocessing
Contains data for 2019 only

In [9]:
source_number = 9
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_9 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

source_9.columns = source_9.columns.str[1:]
source_9.columns = source_9.columns.str[:-1]
source_9.rename(columns={'"Geslacht': 'Geslacht'}, inplace=True)

source_9["Geslacht"] = source_9["Geslacht"].str[1:]
source_9["Geslacht"] = source_9["Geslacht"].str[:-1]

source_9["Leeftijd"] = source_9["Leeftijd"].str[1:]
source_9["Leeftijd"] = source_9["Leeftijd"].str[:-1]

source_9["Soort opname"] = source_9["Soort opname"].str[1:]
source_9["Soort opname"] = source_9["Soort opname"].str[:-1]

source_9["Diagnose"] = source_9["Diagnose"].str[1:]
source_9["Diagnose"] = source_9["Diagnose"].str[:-1]

source_9["Perioden"] = source_9["Perioden"].str[1:]
source_9["Perioden"] = source_9["Perioden"].str[:-1]
source_9["Perioden"] = source_9["Perioden"].str.replace("*","")

source_9["Regio's"] = source_9["Regio's"].str[1:]
source_9["Regio's"] = source_9["Regio's"].str[:-1]

source_9["Opnamen per 10 000 inwoners (per 10 000 inwoners)"] = source_9["Opnamen per 10 000 inwoners (per 10 000 inwoners)"].str[1:]
source_9["Opnamen per 10 000 inwoners (per 10 000 inwoners)"] = source_9["Opnamen per 10 000 inwoners (per 10 000 inwoners)"].str[:-1]

source_9.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

# drop columns with only 1 value
for col in source_9.columns:
    if len(source_9[col].unique()) == 1:
        source_9.drop(col,inplace=True,axis=1)

merged_source_9 = pd.merge(df_final, source_9, on = "Gemeentenaam", how='right')

# some municipalities dont map to provinces, drop those
merged_source_9.drop(merged_source_9[merged_source_9['Provincienaam'].isnull()].index, inplace=True)

merged_source_9['Opnamen per 10 000 inwoners (per 10 000 inwoners)'] = merged_source_9['Opnamen per 10 000 inwoners (per 10 000 inwoners)'].str.replace(',','.')
merged_source_9['Opnamen per 10 000 inwoners (per 10 000 inwoners)'] = pd.to_numeric(merged_source_9['Opnamen per 10 000 inwoners (per 10 000 inwoners)'])

#impute
means = {}
for provincienaam in merged_source_9['Provincienaam'].unique():
    means[provincienaam] = merged_source_9[merged_source_9['Provincienaam'] == provincienaam]['Opnamen per 10 000 inwoners (per 10 000 inwoners)'].mean()

for i, row in merged_source_9[merged_source_9['Opnamen per 10 000 inwoners (per 10 000 inwoners)'].isnull()].iterrows():
    merged_source_9.at[i, 'Opnamen per 10 000 inwoners (per 10 000 inwoners)'] = means[row['Provincienaam']]

# to check:
# merged_source_9.count()

# Translate
merged_source_9.columns = ['Municipality', 'Province', 'Gender', 'Age group', 'Admissions (per 10 000 inhabitants)']
merged_source_9["Gender"].replace({"Mannen": "Men", "Vrouwen": "Women"}, inplace=True)
merged_source_9["Age group"].replace({'0 jaar':'0 years', '1 tot 20 jaar': '1 to 20 years', '20 tot 45 jaar': '20 to 45 years', '45 tot 65 jaar': '45 to 65 years','65 tot 80 jaar': '65 to 80 years', '80 jaar of ouder': '80 years or older'}, inplace=True)

# subject-predicate-object format

s9_dict = {'Subject':[] , 'Predicate': [], 'Object': []}

subjects = merged_source_9['Municipality'].unique()

predicates = [list(merged_source_9.columns)[1]]
genders = list(merged_source_9['Gender'].unique())
ages = list(merged_source_9['Age group'].unique())

for gender in genders:
    for age in ages:
        predicates.append('Admissions (per 10 000 inhabitants)'+', '+gender+', '+age)


for subject in subjects:
    s9_dict['Subject'].append(subject)
    s9_dict['Predicate'].append(predicates[0])
    s9_dict['Object'].append(merged_source_9[merged_source_9['Municipality']==subject]['Province'].unique()[0])
    
    for pedictate in predicates[1:]:
        s9_dict['Subject'].append(subject)
        s9_dict['Predicate'].append(pedictate)
        intermediate = merged_source_9[merged_source_9['Municipality']==subject]
        intermediate = intermediate[intermediate['Gender']==pedictate.split(', ')[1]]
        value = intermediate[intermediate['Age group']==pedictate.split(', ')[2]]['Admissions (per 10 000 inhabitants)']
        s9_dict['Object'].append(float(value))
        
spo_9 = pd.DataFrame(data=s9_dict)
spo_9

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/3849456725.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_9["Perioden"] = source_9["Perioden"].str.replace("*","")


,Subject,Predicate,Object
0,Aa en Hunze,Province,Drenthe
1,Aa en Hunze,"Admissions (per 10 000 inhabitants), Men, 0 years",874.0
2,Aa en Hunze,"Admissions (per 10 000 inhabitants), Men, 1 to...",77.0
3,Aa en Hunze,"Admissions (per 10 000 inhabitants), Men, 20 t...",48.0
4,Aa en Hunze,"Admissions (per 10 000 inhabitants), Men, 45 t...",560.0
...,...,...,...
4896,Zwolle,"Admissions (per 10 000 inhabitants), Women, 1 ...",95.0
4897,Zwolle,"Admissions (per 10 000 inhabitants), Women, 20...",313.0
4898,Zwolle,"Admissions (per 10 000 inhabitants), Women, 45...",87.0
4899,Zwolle,"Admissions (per 10 000 inhabitants), Women, 65...",92.0


### Source 11 preprocessing
Contains data for 2018-2021

In [10]:
source_number = 11
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_11 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

source_11.columns = source_11.columns.str[1:]
source_11.columns = source_11.columns.str[:-1]
source_11.rename(columns={'"Perioden': 'Perioden'}, inplace=True)

source_11["Perioden"] = source_11["Perioden"].str[1:]
source_11["Perioden"] = source_11["Perioden"].str[:-1]
source_11["Perioden"] = source_11["Perioden"].str.replace("*","")

source_11["Regio's"] = source_11["Regio's"].str[1:]
source_11["Regio's"] = source_11["Regio's"].str[:-1]

source_11.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)
merged_source_11 = pd.merge(df_final, source_11, on = "Gemeentenaam", how='right')

merged_source_11.drop(merged_source_11[merged_source_11['Provincienaam'].isnull()].index, inplace=True)

columns = merged_source_11.columns[3:]
for column in columns:
    means = {}
    for provincienaam in merged_source_11['Provincienaam'].unique():
        means[provincienaam] = merged_source_11[merged_source_11['Provincienaam'] == provincienaam][column].mean()
    for i, row in merged_source_11[merged_source_11[column].isnull()].iterrows():
        merged_source_11.at[i, column] = means[row['Provincienaam']]

# compute mean for all years
condition = merged_source_11.groupby(['Gemeentenaam', 'Provincienaam'])
merged_source_11 = condition.mean(['Totaal alle onderliggende doodsoorzaken (aantal)', 'Nieuwvormingen (aantal)', 'Ziekten van hart en vaatstelsel (aantal)', 'Ziekten van ademhalingsstelsel (aantal)']).reset_index()

# Translate
merged_source_11.columns = ['Municipality', 'Province','Total all underlying causes of death (number)','New formations (number)', 'Diseases of the cardiovascular system (number)','Diseases of the respiratory system (number)']

spo_11 = merged_source_11.melt(id_vars=['Municipality'], var_name='Predicate', value_name='Object')
spo_11.rename(columns={"Municipality": "Subject"}, inplace = True)
spo_11

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/712242594.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_11["Perioden"] = source_11["Perioden"].str.replace("*","")


,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
1,Aa en Hunze,Province,Drenthe
2,Aalburg,Province,Noord-Brabant
3,Aalsmeer,Province,Noord-Holland
4,Aalten,Province,Gelderland
...,...,...,...
1880,Zundert,Diseases of the respiratory system (number),20.0
1881,Zutphen,Diseases of the respiratory system (number),47.666667
1882,Zwartewaterland,Diseases of the respiratory system (number),11.333333
1883,Zwijndrecht,Diseases of the respiratory system (number),32.0


### Source 14 preprocessing
Contains data for 2019, 2020 and 2021 only

In [11]:
source_number = 14
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_14 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

source_14.columns = source_14.columns.str[1:]
source_14.columns = source_14.columns.str[:-1]
source_14.rename(columns={'"Perioden': 'Perioden'}, inplace=True)

source_14["Perioden"] = source_14["Perioden"].str[1:]
source_14["Perioden"] = source_14["Perioden"].str[:-1]
source_14["Perioden"] = source_14["Perioden"].str.replace("*","")

source_14["Regio's"] = source_14["Regio's"].str[1:]
source_14["Regio's"] = source_14["Regio's"].str[:-1]

source_14.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)
merged_source_14 = pd.merge(df_final, source_14, on = "Gemeentenaam", how='right')

merged_source_14.drop(merged_source_14[merged_source_14['Provincienaam'].isnull()].index, inplace=True)

merged_source_14['Bevolkingsgroei/Bevolkingsgroei, relatief (%)'] = merged_source_14['Bevolkingsgroei/Bevolkingsgroei, relatief (%)'].str.replace(',','.')
merged_source_14['Bevolkingsgroei/Bevolkingsgroei, relatief (%)'] = pd.to_numeric(merged_source_14['Bevolkingsgroei/Bevolkingsgroei, relatief (%)'])

columns = merged_source_14.columns[3:]
for column in columns:
    means = {}
    for provincienaam in merged_source_14['Provincienaam'].unique():
        means[provincienaam] = merged_source_14[merged_source_14['Provincienaam'] == provincienaam][column].mean()
    for i, row in merged_source_14[merged_source_14[column].isnull()].iterrows():
        merged_source_14.at[i, column] = means[row['Provincienaam']]

# compute mean for all years
condition = merged_source_14.groupby(['Gemeentenaam', 'Provincienaam'])
merged_source_14 = condition.mean(['Bevolking aan het begin van de periode (aantal)', 'Levend geboren kinderen (aantal)', 'Overledenen (aantal)', 'Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal)', 'Bevolkingsgroei/Bevolkingsgroei, relatief (%)']).reset_index()

# Translate
merged_source_14.columns = ['Municipality','Province','Population','LiveBornChildren','Deceased','DepartureToAnotherMunicipality','PopulationGrowth']

spo_14 = merged_source_14.melt(id_vars=['Municipality'], var_name='Predicate', value_name='Object')
spo_14.rename(columns={"Municipality": "Subject"}, inplace = True)
spo_14

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/1082819723.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_14["Perioden"] = source_14["Perioden"].str.replace("*","")


,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
1,Aa en Hunze,Province,Drenthe
2,Aalburg,Province,Noord-Brabant
3,Aalsmeer,Province,Noord-Holland
4,Aalten,Province,Gelderland
...,...,...,...
2263,Zundert,PopulationGrowth,0.983333
2264,Zutphen,PopulationGrowth,0.5
2265,Zwartewaterland,PopulationGrowth,0.753333
2266,Zwijndrecht,PopulationGrowth,0.096667


### Combine all the long formats into one huge table

In [12]:
# TODO: COMBINE THEM BUT FIRST DISCUSS NAMING CONVENTIONS AND ALSO DROP DUPLICATES? SHOULDNT BE THERE RIGHT OR SORT VALUES BY SUBJECT DO WE WANT THAT? BCS
# NOW WE KNOW WHICH ROWS COME FROM WHICH TABLE
# TODO: DISCUSS OPTIMIZATION OF CODE -> LONG FORMAT CAN BE DONE EASIER
KE_TABLE = pd.concat([spo_6,spo_7,spo_9,spo_11,spo_14]).drop_duplicates().reset_index(drop=True).sort_values(by=['Subject'])
KE_TABLE

,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
9147,'s-Hertogenbosch,"Admissions (per 10 000 inhabitants), Men, 65 t...",355.0
9148,'s-Hertogenbosch,"Admissions (per 10 000 inhabitants), Men, 80 y...",552.0
9149,'s-Hertogenbosch,"Admissions (per 10 000 inhabitants), Women, 0 ...",431.0
9150,'s-Hertogenbosch,"Admissions (per 10 000 inhabitants), Women, 1 ...",34.0
...,...,...,...
7428,Zwolle,"Persons with medicines - relative (%), Men, 55...",5.05
7427,Zwolle,"Persons with medicines - relative (%), Men, 45...",5.857199
7426,Zwolle,"Persons with medicines - relative (%), Men, 35...",7.45
14226,Zwolle,LiveBornChildren,1460.333333


### Obtain rdf file for visualization

### Example using source 14

In [13]:
# get a subset of the triples
test = merged_source_14[merged_source_14['Province']=='Noord-Brabant']
test = test.melt(id_vars=['Municipality'], var_name='Predicate', value_name='Object')
test.rename(columns={"Municipality": "Subject"}, inplace = True)
test

,Subject,Predicate,Object
0,'s-Hertogenbosch,Province,Noord-Brabant
1,Aalburg,Province,Noord-Brabant
2,Alphen-Chaam,Province,Noord-Brabant
3,Altena,Province,Noord-Brabant
4,Asten,Province,Noord-Brabant
...,...,...,...
385,Waalwijk,PopulationGrowth,0.763333
386,Werkendam,PopulationGrowth,0.641135
387,Woensdrecht,PopulationGrowth,0.393333
388,Woudrichem,PopulationGrowth,0.641135


In [14]:
# remove all problematic characters
test['Object'] = test['Object'].astype(str)
test['Object'] = test['Object'].str.replace(' ','-')
test['Subject'] = test['Subject'].str.replace(' ','-')
test['Subject'] = test['Subject'].str.replace("'","")
test['Subject'] = test['Subject'].str.replace("(","")
test['Subject'] = test['Subject'].str.replace(")","")

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/86439305.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test['Subject'] = test['Subject'].str.replace("(","")
C:\Users\t1rk9\AppData\Local\Temp/ipykernel_2748/86439305.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test['Subject'] = test['Subject'].str.replace(")","")


In [15]:
# obtain RDF data
from rdflib import Graph, URIRef, Namespace

g = Graph()
n = Namespace('ex:')

for inded, row in test.iterrows():
    # add triple to rdf-graph
    g.add((URIRef(n+row["Subject"]), URIRef(n+row["Predicate"]), URIRef(n+row["Object"])))

ttl = g.serialize(format = 'turtle')
with open('example.ttl', 'w') as file:
    file.write(ttl)

In [16]:
# upload file to be visualized in this link:
# https://issemantic.net/rdf-visualizer